# Generate HOG Feature from given input file

In [1]:
#Import libraries

import pandas as pd
import numpy as np
from skimage.feature import hog

In [2]:
df = pd.read_csv("input.csv")

In [3]:
ppc = 4
hog_images = []
hog_features = []
for image in df['pixels']:
    image = np.fromstring(image,dtype='int',sep=' ').reshape(48,48).astype('uint8')
    fd,hog_image = hog(image, orientations=9, pixels_per_cell=(ppc,ppc),cells_per_block=(1, 1),block_norm= 'L2',visualise=True)
    hog_images.append(hog_image)
    hog_features.append(fd.tolist())

C:\Users\user\Anaconda3\lib\site-packages\skimage\feature\_hog.py:248: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


In [4]:
df['hog_feature'] = hog_features

In [5]:
#Generate training csv file
df_train = df[df['Usage'] == 'Training']
df_train.to_csv("output/train_hog.csv")

#Generate test csv file
df_test = df[~df.isin(df_train).all(1)]
df_test.to_csv("output/test_hog.csv")

In [7]:
# Generate X_train, Y_train

X_train = []
train_hog = np.array(df_train['hog_feature'])
for each in train_hog:
    each = np.array(each)
    X_train.append(each)

#X_train
X_train = np.array(X_train)

Y_train = np.array(df_train['emotion (label)'])
#Y_train
Y_train = Y_train.reshape(len(Y_train),1)

In [8]:
# Generate x_test, y_test

x_test = []
test_hog = np.array(df_test['hog_feature'])
for each in test_hog:
    each = np.array(each)
    x_test.append(each)

#X_train
x_test = np.array(x_test)

y_test = np.array(df_test['emotion (label)'])
#Y_train
y_test = y_test.reshape(len(y_test),1)

# Training Model with SVM

In [9]:
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score, precision_score, recall_score, f1_score

In [10]:
#Initiate svm and train with training feature and label

clf = svm.SVC()
clf.fit(X_train,Y_train)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [11]:
#Predict label for test feature

y_pred = clf.predict(x_test)

In [38]:
df_test['Row Number'] = df_test.index

df_test['pred_label'] = y_pred

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [43]:
#creating a df with columns predicted label and Row Number from input.csv(please note that indexing start from 0 and not 1)
df_pred = df_test[['pred_label','Row Number']]

In [45]:
#saving csv of predicted labels

df_pred.to_csv("output\predicted_label.csv")

# Evaluation of Model

In [19]:
# Evaluation on test data


pred_scr = precision_score(y_test, y_pred)
recall_scr = recall_score(y_test, y_pred)
f1_scr = f1_score(y_test, y_pred)
acr_score = accuracy_score(y_test, y_pred)

In [21]:
print(F"{ pred_scr }, { recall_scr }, { f1_scr }, { acr_score }")
print(classification_report(y_test, y_pred))

0.8018539976825029, 0.3900789177001127, 0.5248388320060675, 0.8254388409027584
              precision    recall  f1-score   support

           0       0.83      0.97      0.89      5404
           1       0.80      0.39      0.52      1774

   micro avg       0.83      0.83      0.83      7178
   macro avg       0.82      0.68      0.71      7178
weighted avg       0.82      0.83      0.80      7178



In [70]:
with open("output/test_evaluation.txt","a") as file:
    file.write(F" Precision score is -> { pred_scr } \n")
    file.write(F" Recall score is -> { recall_scr } \n")
    file.write(F" F1 score is -> { f1_scr } \n")
    file.write(F" Acuuracy score is -> { acr_score } \n")
    file.write(F"Classification report is below \n\n\n { classification_report(y_test, y_pred) }")

In [59]:
#train evaluation

#shuffle train_set and get 20 percentage of training data for evaluation, i am calling it as val

from sklearn.utils import shuffle
X_train, Y_train = shuffle(X_train, Y_train, random_state=15)

X_val = X_train[:int(len(X_train) * 20 / 100 )]
Y_val_true = Y_train[:int(len(Y_train) * 20 / 100 )]

In [63]:
Y_val_pred = clf.predict(X_val)

In [64]:
pred_scr_val = precision_score(Y_val_true, Y_val_pred)
recall_scr_val = recall_score(Y_val_true, Y_val_pred)
f1_scr_val = f1_score(Y_val_true, Y_val_pred)
acr_score_val = accuracy_score(Y_val_true, Y_val_pred)

In [65]:
print(F"{ pred_scr_val }, { recall_scr_val }, { f1_scr_val }, { acr_score_val }")
print(classification_report(Y_val_true, Y_val_pred))

0.8310055865921788, 0.40669856459330145, 0.5461220743460303, 0.8277303605643616
              precision    recall  f1-score   support

           0       0.83      0.97      0.89      4278
           1       0.83      0.41      0.55      1463

   micro avg       0.83      0.83      0.83      5741
   macro avg       0.83      0.69      0.72      5741
weighted avg       0.83      0.83      0.81      5741



In [71]:
with open("output/train_evaluation.txt","a") as file:
    file.write(F" Precision score is -> { pred_scr_val } \n")
    file.write(F" Recall score is -> { recall_scr_val } \n")
    file.write(F" F1 score is -> { f1_scr_val } \n")
    file.write(F" Acuuracy score is -> { acr_score_val } \n")
    file.write(F"Classification report is below \n\n\n { classification_report(Y_val_true, Y_val_pred) }")

In [67]:
avg_precision = ( pred_scr + pred_scr_val ) / 2
avg_recall = ( recall_scr / recall_scr_val ) / 2
avg_f1scr = ( f1_scr / f1_scr_val ) / 2
avg_accr = ( acr_score + acr_score_val ) / 2

print(F"{ avg_precision }, { avg_recall }, { avg_f1scr }, { avg_accr }")

0.8164297921373409, 0.47956761058425623, 0.48051420795849625, 0.82658460073356


In [72]:
with open("output/train_evaluation.txt","a") as file:
    file.write(F"AVERAGE EVALUATION REPORT -> \n\n\n\n")
    file.write(F" Average Precision score is -> { avg_precision } \n")
    file.write(F" Average Recall score is -> { avg_recall } \n")
    file.write(F" Average F1 score is -> { avg_f1scr } \n")
    file.write(F" Average Acuuracy score is -> { avg_accr }")

In [73]:
with open("output/test_evaluation.txt","a") as file:
    file.write(F"AVERAGE EVALUATION REPORT -> \n\n\n\n")
    file.write(F" Average Precision score is -> { avg_precision } \n")
    file.write(F" Average Recall score is -> { avg_recall } \n")
    file.write(F" Average F1 score is -> { avg_f1scr } \n")
    file.write(F" Average Acuuracy score is -> { avg_accr }")